In [1]:
import tweepy
import textblob
import time
from datetime import datetime
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
import matplotlib.pyplot as plt
import requests, json
from time import sleep

### TWITTER SENTIMENT ANALYSIS

In [2]:
class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # keys and tokens from the Twitter Dev Console
        consumer_key = 'vU0U124x884mQKjXP0XVUyeOV'
        consumer_secret = 'PHnqYoo0bFvA3IRbx6y4Ku0OVDxGF0Gz1bfAV8EDJAJ6DD1Cpq'
        access_token = '1327003820359118848-ecWID7cDZmq8yRWRanh5kB77wlk91M'
        access_token_secret = 'chjFwYwMykw4P8xMwjrR3nXRg7irWuQbhJqlX69gCTxja'
  

        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")

    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'

    def tweet_file(self):
        self.message="Sentiment Analysis by @BlockchainEng"
        self.filename='SentimentAnalysis.png'
        print(self)
        self.api.update_with_media(message = self.message, filename= self.filename)

    def get_tweets(self, query, count = 10):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []

        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(q = query, count = count)

            # parsing tweets one by one
            list_of_tweets = []
            for tweet in fetched_tweets:

                # empty dictionary to store required params of a tweet
                parsed_tweet = {}

                # saving text of tweet
                #tweet1 =
                #tweet1=TwitterClient.api.clean_tweet(tweet_text)
                parsed_tweet['text'] = tweet.text
                list_of_tweets.append(tweet.text)
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)

                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)

            # return parsed tweets
            with open('TweetHistory.txt', 'a+') as f1:
                for tweet in list_of_tweets:
                    try:
                        f1.write(tweet)
                    except:
                        pass

            return tweets

        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))


def main():
    # creating object of TwitterClient Class
    api = TwitterClient()
    # calling function to get tweets
    search_term = 'Bitcoin'
    list_coins=['Bitcoin'] #, 'Ethereum', 'EOS', 'Cryptocurrency', 'Blockchain', 'BTC'] #'Donald Trump', 'Barack Obama', 'George Washington']
    count1=200
    list_coin_val = []
    start_time = datetime.now()
    try:
        for i in range(0,1):
            #This For Loop controls how many data 'cycles' are collected before visualization and tweeting
            for coin in list_coins:
                tweets = api.get_tweets(query = coin, count = count1)

                # picking positive tweets from tweets
                ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
                # percentage of positive tweets
                print("\n\nTime: {}".format(str(datetime.now())))
                print("Sentiment Values for {}\nNumber of Tweets Analyzed: {}".format(coin, count1))
                positive_tweet_percentage = round(100*len(ptweets)/len(tweets),6)
                print("Positive tweets percentage: {} %".format(positive_tweet_percentage))
                # picking negative tweets from tweets
                ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
                # percentage of negative tweets
                negative_tweet_percentage = round(100*len(ntweets)/len(tweets),6)
                print("Negative tweets percentage: {} %".format(negative_tweet_percentage))
                # percentage of neutral tweets
                #neutral_tweet_percentage = round(100*(len(tweets) - len(ntweets) - len(ptweets))/len(tweets),6)
                #print("Neutral tweets percentage: {} % \
                    #".format(neutral_tweet_percentage))
                #current_price = getBitcoinPrice()
                #list_coin_val.append([coin, positive_tweet_percentage, negative_tweet_percentage, neutral_tweet_percentage, str(datetime.now()), current_price])
                #save_to_file(coin, positive_tweet_percentage, negative_tweet_percentage, neutral_tweet_percentage, datetime.now(), current_price, "SentimentHistorical.csv")
                #print("Current Price: {}".format(getBitcoinPrice()))
                #trading(current_price, positive_tweet_percentage, negative_tweet_percentage)    #used for Trading Signals
        #end_time = datetime.now()
        #print(list_coin_val, start_time, end_time)
        return positive_tweet_percentage, negative_tweet_percentage
    except:
        print("ERROR - COLLECTING DATA")

In [3]:
def save_to_file(coin, positive_tweet_percentage, negative_tweet_percentage, neutral_tweet_percentage, time, current_price, filename = "SentimentHistorical.csv"):
    with open(filename, 'a+') as  f:
        line = coin, positive_tweet_percentage, negative_tweet_percentage, neutral_tweet_percentage, time, current_price
        f.writelines(str(line) + '\n')

### TRADE & VISUALIZATION

In [4]:
def getBitcoinPrice():
    URL = 'https://www.bitstamp.net/api/ticker/'
    try:
        r = requests.get(URL)
        priceFloat = float(json.loads(r.text)['last'])
        return priceFloat
    except requests.ConnectionError:
        print("Error querying Bitstamp API")
        
def tweet_file(self, message, filename):
    self.api.update_with_media(message, filename)
    
def historic_data_viz(self):
    #Load Historic Data and Visual into graph form for entire recorded amount
    historic_data_list = []
    i=0
    with open('SentimentHistorical.csv') as f1:
        lines = list(f1.readlines())
        for line in lines:
            print(line)
            data=list(line.split(','))
            print(data)
            if i==0:
                start_time=data[5]
            if i==len(f1.readlines()):
                end_time=data[5]
            historic_data_list.append(data)
            i+=1
    data_visualize(historic_data_list, start_time, end_time, self)

def trading(current_price, positive_sentiment_percent, negative_sentiment_percent):
    print("Positive Sentiment - Negative Sentiment (Net Positive)", positive_sentiment_percent-negative_sentiment_percent)
    if positive_sentiment_percent>1.5*negative_sentiment_percent:
        #Buy
        #use Khal's Code to place order, entries & exits
        print("TEST - BUY SIGNAL")
        pass
    if negative_sentiment_percent>1.5*positive_sentiment_percent:
        #Sell
        print("TEST - SELL SIGNAL")
        #use Khal's Code to place order, entries & exits
        pass

In [5]:
positive_tweet_percentage, negative_tweet_percentage = main()



Time: 2021-06-11 22:14:54.778597
Sentiment Values for Bitcoin
Number of Tweets Analyzed: 200
Positive tweets percentage: 36.781609 %
Negative tweets percentage: 10.344828 %


## Bollinger Bands

In [6]:
# Code Ref: https://github.com/IgorWounds/Kraken-API---A-Complete-Guide-Algotrading101/blob/main/Kraken%20Public%20Endpoints.ipynb

In [7]:
# Initial imports
from time import sleep
import krakenex
import requests
import pandas as pd
from pykrakenapi import KrakenAPI
import plotly.graph_objects as go
import plotly.express as px

/Users/coneychiwabee/opt/anaconda3/envs/pyvizenv/lib/python3.7/site-packages/xarray/core/merge.py:10: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [8]:
# Set API keys and secret
api = krakenex.API()
k = KrakenAPI(api)
#api.load_key('KrakenPass.txt')

In [9]:
# Get asset info
data = k.get_asset_info()

In [10]:
# Get prices
ticker = k.get_ticker_information('BTCUSD')

public call frequency exceeded (seconds=0.253886) 
 sleeping for 5 seconds


In [11]:
data1 = requests.get('https://api.kraken.com/0/public/Assets').json()

## 1. Historical Data

In [12]:
ohlc = k.get_ohlc_data('BTCUSD', interval=1440, ascending = True)
# Drop NAs (Should we replace NAs?)
ohlc[0]['close'] = ohlc[0]['close'].dropna()
#ohlc[0]

public call frequency exceeded (seconds=0.347482) 
 sleeping for 5 seconds


# Trading Signal Features

---

### Import Libraries and Dependencies

In [13]:
# Import libraries and dependencies
import pandas as pd
import numpy as np
from pathlib import Path
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

### Construct Bollinger Band (Trading Signal)

In [14]:
# Set bollinger band window
bollinger_window = 20

# Calculate rolling mean and standard deviation
ohlc[0]['bollinger_mid_band'] = ohlc[0]['close'].rolling(window=bollinger_window).mean()
ohlc[0]['bollinger_std'] = ohlc[0]['close'].rolling(window=20).std()

# Calculate upper and lowers bands of bollinger band
ohlc[0]['bollinger_upper_band']  = ohlc[0]['bollinger_mid_band'] + (ohlc[0]['bollinger_std'] * 1)
ohlc[0]['bollinger_lower_band']  = ohlc[0]['bollinger_mid_band'] - (ohlc[0]['bollinger_std'] * 1)

# Calculate bollinger band trading signal
ohlc[0]['bollinger_long'] = np.where(ohlc[0]['close'] < ohlc[0]['bollinger_lower_band'], 1.0, 0.0)
ohlc[0]['bollinger_short'] = np.where(ohlc[0]['close'] > ohlc[0]['bollinger_upper_band'], -1.0, 0.0)
ohlc[0]['bollinger_signal'] = ohlc[0]['bollinger_long'] + ohlc[0]['bollinger_short']
#ohlc[0]['bollinger_signal'].tail()

### Plot Bollinger Bands

In [15]:
# Plot the Bollinger Bands for BTC/USD closing prices
#ohlc[0][['close','bollinger_mid_band','bollinger_upper_band','bollinger_lower_band']].plot(figsize=(20,10))

In [16]:
# bollinger signal 1 = buy, 0 = hold, -1 = sell
# sentiment signal 1 = buy, 0 = hold, -1 = sell
if positive_tweet_percentage > 50:
    sentiment_signal = 1
elif negative_tweet_percentage > 50:
    sentiment_signal = -1
else: sentiment_signal = 0

bollinger_signal = ohlc[0]['bollinger_signal'][-1]

def get_genie_decision(bollinger_signal, sentiment_signal):
    if bollinger_signal == 1 and sentiment_signal == 1:
        print("IT’S A GOOD TIME TO BUY! WHY? BECAUSE BITCOIN IS UNDERVALUED AND TWITTER IS BULLISH ON BITCOIN!")
    if bollinger_signal == 1 and sentiment_signal == 0:
        print("IT’S A GOOD TIME TO BUY! WHY? BECAUSE BITCOIN IS UNDERVALUED AND TWITTER IS NEUTRAL ON BITCOIN!")
    if bollinger_signal == 1 and sentiment_signal == -1:
        print("IT’S A GOOD TIME TO SELL! WHY? BECAUSE EVEN THOUGH BITCOIN IS UNDERVALUED, TWITTER IS BEARISH ON BITCOIN!")
    if bollinger_signal == 0 and sentiment_signal == 1:
        print("IT’S A GOOD TIME TO BUY! WHY? BECAUSE EVEN THOUGH BITCOIN IS NOT UNDERVALUED, TWITTER IS BULLISH ON BITCOIN!")
    if bollinger_signal == 0 and sentiment_signal == 0:
        print("IT’S A GOOD TIME TO HOLD! WHY? BECAUSE BITCOIN IS NEITHER OVERVALUED NOR UNDERVALUED AND TWITTER IS NEUTRAL ON BITCOIN!")
    if bollinger_signal == 0 and sentiment_signal == -1:
        print("IT’S A GOOD TIME TO SELL! WHY? BECAUSE EVEN THOUGH BITCOIN IS NOT OVERVALUED, TWITTER IS BEARISH ON BITCOIN!")
    if bollinger_signal == -1 and sentiment_signal == 1:
        print("IT’S A GOOD TIME TO BUY! WHY? BECAUSE EVEN THOUGH BITCOIN IS OVERVALUED, TWITTER IS BULLISH ON BITCOIN!")
    if bollinger_signal == -1 and sentiment_signal == 0:
        print("IT’S A GOOD TIME TO SELL! WHY? BECAUSE BITCOIN IS OVERVALUED AND TWITTER IS NEUTRAL ON BITCOIN!")
    if bollinger_signal == -1 and sentiment_signal == -1:
        print("IT’S A GOOD TIME TO SELL! WHY? BECAUSE BITCOIN IS OVERVALUED AND TWITTER IS BEARISH ON BITCOIN!")

#print("bollinger signal =" , bollinger_signal, "sentiment signal =" , sentiment_signal)
get_genie_decision(bollinger_signal, sentiment_signal)

IT’S A GOOD TIME TO HOLD! WHY? BECAUSE BITCOIN IS NEITHER OVERVALUED NOR UNDERVALUED AND TWITTER IS NEUTRAL ON BITCOIN!
